# Sanity Check: Local Inference

You need to generate predictions so you can test the model.  Axolotl uploaded the trained model to

[parlance-labs/hc-mistral-alpaca](https://huggingface.co/parlance-labs/hc-mistral-alpaca)

In [1]:
import torch, gc, inspect, transformers

from peft import AutoPeftModelForCausalLM, PeftConfig, PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
def cleanup():
    """Free up memory and reset stats"""
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats(device)

# cleanup()

In [3]:
device = "cuda"

def print_memory_stats():
    """Print two different measures of GPU memory usage"""
    print(f"Max memory allocated: {torch.cuda.max_memory_allocated(device)/1e9:.2f}GB")
    # reserved (aka 'max_memory_cached') is ~the allocated memory plus pre-cached memory
    print(f"Max memory reserved: {torch.cuda.max_memory_reserved(device)/1e9:.2f}GB") 

print_memory_stats()

Max memory allocated: 0.00GB
Max memory reserved: 0.00GB


### Optional - Load Model Locally and push to HF-hub

In [2]:
model_id='qlora-alpaca-out'
model = AutoPeftModelForCausalLM.from_pretrained(model_id, load_in_4bit=True).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
This can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

push to HF hub:

In [13]:
HF_REPO = "jermyn/CodeQwen1.5-7B-Chat-NLQ2Cypher"

model.push_to_hub(HF_REPO)

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/1.83G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jermyn/CodeQwen1.5-7B-Chat-NLQ2Cypher/commit/4d7c3ec998109fee8591203dde7a5f78b9540277', commit_message='Upload model', commit_description='', oid='4d7c3ec998109fee8591203dde7a5f78b9540277', pr_url=None, pr_revision=None, pr_num=None)

### Load from HF hub:

In [4]:
model_peft_id="jermyn/CodeQwen1.5-7B-Chat-NLQ2Cypher"
# model_peft_id="jermyn/deepseek-code-1.3b-inst-NLQ2Cypher"

# model_base_id="Qwen/CodeQwen1.5-7B-Chat"

model = AutoPeftModelForCausalLM.from_pretrained(model_peft_id, load_in_4bit=True).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_peft_id)
tokenizer.pad_token = tokenizer.eos_token

# ALTERNATIVE
# load base
# base_model = AutoModelForCausalLM.from_pretrained(model_base_id)
# tokenizer = AutoTokenizer.from_pretrained(model_peft_id)
# resize
# base_model.resize_token_embeddings(len(tokenizer))

# Merge Model with Adapter
# model = PeftModel.from_pretrained(base_model, model_peft_id)
# tokenizer.pad_token = tokenizer.eos_token

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
This can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# config = PeftConfig.from_pretrained(model_peft_id)
# print(config)

### Prompt Template

Next, we have to construct a prompt template that is as close as possible to the prompt template we saw earlier.

In [5]:
def prompt(nlq, schema):
    return f"""You are an assistant that takes a natural language query (NLQ) and a graph database schema to produce a Neo4J Cypher query.

    ### Instruction:

    NLQ: "{nlq}" 
    
    Schema: {schema}

    ### Response:
    """


def prompt_tok(nlq, schema, return_ids=False):
    _p = prompt(nlq, schema)
    input_ids = tokenizer(_p, return_tensors="pt", truncation=True).input_ids.cuda()
    out_ids = model.generate(
        input_ids=input_ids, max_new_tokens=5000, do_sample=False
    )
    ids = out_ids.detach().cpu().numpy()
    if return_ids: 
        return out_ids
    return tokenizer.batch_decode(ids, skip_special_tokens=True)[0][len(_p):]

## Sanity Check Examples

Next, we sanity check a few examples to make sure that:

- Our prompt template is constructed correctly (indeed I made some mistakes at first!)
- The model is working as expected.

In [13]:
eval_dict = {
    "nlq_1": "Finds all phone calls lasting more than 60 minutes",
    "schema_1": 'Node properties are the following: ":Person {surname: STRING, nhs_no: STRING, name: STRING, age: STRING},:Location {latitude: FLOAT, postcode: STRING, longitude: FLOAT, address: STRING},:Phone {phoneNo: STRING},:Email {email_address: STRING},:Officer {badge_no: STRING, rank: STRING, name: STRING, surname: STRING},:PostCode {code: STRING},:Area {areaCode: STRING},:PhoneCall {call_duration: STRING, call_time: STRING, call_date: STRING, call_type: STRING},:Crime {date: STRING, id: STRING, type: STRING, last_outcome: STRING, note: STRING, charge: STRING},:Object {description: STRING, id: STRING, type: STRING},:Vehicle {model: STRING, reg: STRING, make: STRING, year: STRING}" Relationship properties are the following: ":CURRENT_ADDRESS {},:HAS_PHONE {},:HAS_EMAIL {},:HAS_POSTCODE {},:POSTCODE_IN_AREA {},:LOCATION_IN_AREA {},:KNOWS_SN {},:KNOWS {},:CALLER {},:CALLED {},:KNOWS_PHONE {},:OCCURRED_AT {},:INVESTIGATED_BY {},:INVOLVED_IN {},:PARTY_TO {},:FAMILY_REL {rel_type: STRING},:KNOWS_LW {}" Relationship point from source to target nodes "(:Person)-[:FAMILY_REL]->(:Person),(:Person)-[:CURRENT_ADDRESS]->(:Location),(:Person)-[:KNOWS]->(:Person),(:Person)-[:HAS_PHONE]->(:Phone),(:Person)-[:KNOWS_PHONE]->(:Person),(:Person)-[:HAS_EMAIL]->(:Email),(:Person)-[:KNOWS_SN]->(:Person),(:Person)-[:KNOWS_LW]->(:Person),(:Person)-[:PARTY_TO]->(:Crime),(:Location)-[:LOCATION_IN_AREA]->(:Area),(:Location)-[:HAS_POSTCODE]->(:PostCode),(:PostCode)-[:POSTCODE_IN_AREA]->(:Area),(:PhoneCall)-[:CALLED]->(:Phone),(:PhoneCall)-[:CALLER]->(:Phone),(:Crime)-[:INVESTIGATED_BY]->(:Officer),(:Crime)-[:OCCURRED_AT]->(:Location),(:Object)-[:INVOLVED_IN]->(:Crime),(:Vehicle)-[:INVOLVED_IN]->(:Crime)"',
    "nlq_2": "Find cars involved in crime with ID '%crime_id%'.",
    "schema_2": 'Node properties are the following: ":Person {surname: STRING, nhs_no: STRING, name: STRING, age: STRING},:Location {latitude: FLOAT, postcode: STRING, longitude: FLOAT, address: STRING},:Phone {phoneNo: STRING},:Email {email_address: STRING},:Officer {badge_no: STRING, rank: STRING, name: STRING, surname: STRING},:PostCode {code: STRING},:Area {areaCode: STRING},:PhoneCall {call_duration: STRING, call_time: STRING, call_date: STRING, call_type: STRING},:Crime {date: STRING, id: STRING, type: STRING, last_outcome: STRING, note: STRING, charge: STRING},:Object {description: STRING, id: STRING, type: STRING},:Vehicle {model: STRING, reg: STRING, make: STRING, year: STRING}" Relationship properties are the following: ":CURRENT_ADDRESS {},:HAS_PHONE {},:HAS_EMAIL {},:HAS_POSTCODE {},:POSTCODE_IN_AREA {},:LOCATION_IN_AREA {},:KNOWS_SN {},:KNOWS {},:CALLER {},:CALLED {},:KNOWS_PHONE {},:OCCURRED_AT {},:INVESTIGATED_BY {},:INVOLVED_IN {},:PARTY_TO {},:FAMILY_REL {rel_type: STRING},:KNOWS_LW {}" Relationship point from source to target nodes "(:Person)-[:FAMILY_REL]->(:Person),(:Person)-[:CURRENT_ADDRESS]->(:Location),(:Person)-[:KNOWS]->(:Person),(:Person)-[:HAS_PHONE]->(:Phone),(:Person)-[:KNOWS_PHONE]->(:Person),(:Person)-[:HAS_EMAIL]->(:Email),(:Person)-[:KNOWS_SN]->(:Person),(:Person)-[:KNOWS_LW]->(:Person),(:Person)-[:PARTY_TO]->(:Crime),(:Location)-[:LOCATION_IN_AREA]->(:Area),(:Location)-[:HAS_POSTCODE]->(:PostCode),(:PostCode)-[:POSTCODE_IN_AREA]->(:Area),(:PhoneCall)-[:CALLED]->(:Phone),(:PhoneCall)-[:CALLER]->(:Phone),(:Crime)-[:INVESTIGATED_BY]->(:Officer),(:Crime)-[:OCCURRED_AT]->(:Location),(:Object)-[:INVOLVED_IN]->(:Crime),(:Vehicle)-[:INVOLVED_IN]->(:Crime)"',
    "nlq_3": "List all persons who know someone named Abi.",
    "schema_3": 'Node properties are the following: ":Person {surname: STRING, nhs_no: STRING, name: STRING, age: STRING},:Location {latitude: FLOAT, postcode: STRING, longitude: FLOAT, address: STRING},:Phone {phoneNo: STRING},:Email {email_address: STRING},:Officer {badge_no: STRING, rank: STRING, name: STRING, surname: STRING},:PostCode {code: STRING},:Area {areaCode: STRING},:PhoneCall {call_duration: STRING, call_time: STRING, call_date: STRING, call_type: STRING},:Crime {date: STRING, id: STRING, type: STRING, last_outcome: STRING, note: STRING, charge: STRING},:Object {description: STRING, id: STRING, type: STRING},:Vehicle {model: STRING, reg: STRING, make: STRING, year: STRING}" Relationship properties are the following: ":CURRENT_ADDRESS {},:HAS_PHONE {},:HAS_EMAIL {},:HAS_POSTCODE {},:POSTCODE_IN_AREA {},:LOCATION_IN_AREA {},:KNOWS_SN {},:KNOWS {},:CALLER {},:CALLED {},:KNOWS_PHONE {},:OCCURRED_AT {},:INVESTIGATED_BY {},:INVOLVED_IN {},:PARTY_TO {},:FAMILY_REL {rel_type: STRING},:KNOWS_LW {}" Relationship point from source to target nodes "(:Person)-[:FAMILY_REL]->(:Person),(:Person)-[:CURRENT_ADDRESS]->(:Location),(:Person)-[:KNOWS]->(:Person),(:Person)-[:HAS_PHONE]->(:Phone),(:Person)-[:KNOWS_PHONE]->(:Person),(:Person)-[:HAS_EMAIL]->(:Email),(:Person)-[:KNOWS_SN]->(:Person),(:Person)-[:KNOWS_LW]->(:Person),(:Person)-[:PARTY_TO]->(:Crime),(:Location)-[:LOCATION_IN_AREA]->(:Area),(:Location)-[:HAS_POSTCODE]->(:PostCode),(:PostCode)-[:POSTCODE_IN_AREA]->(:Area),(:PhoneCall)-[:CALLED]->(:Phone),(:PhoneCall)-[:CALLER]->(:Phone),(:Crime)-[:INVESTIGATED_BY]->(:Officer),(:Crime)-[:OCCURRED_AT]->(:Location),(:Object)-[:INVOLVED_IN]->(:Crime),(:Vehicle)-[:INVOLVED_IN]->(:Crime)"',
    "nlq_4": "Find all Officers whose names contain 'Alex', along with their associated entities",
    "schema_4": 'Node properties are the following: ":Person {surname: STRING, nhs_no: STRING, name: STRING, age: STRING},:Location {latitude: FLOAT, postcode: STRING, longitude: FLOAT, address: STRING},:Phone {phoneNo: STRING},:Email {email_address: STRING},:Officer {badge_no: STRING, rank: STRING, name: STRING, surname: STRING},:PostCode {code: STRING},:Area {areaCode: STRING},:PhoneCall {call_duration: STRING, call_time: STRING, call_date: STRING, call_type: STRING},:Crime {date: STRING, id: STRING, type: STRING, last_outcome: STRING, note: STRING, charge: STRING},:Object {description: STRING, id: STRING, type: STRING},:Vehicle {model: STRING, reg: STRING, make: STRING, year: STRING}" Relationship properties are the following: ":CURRENT_ADDRESS {},:HAS_PHONE {},:HAS_EMAIL {},:HAS_POSTCODE {},:POSTCODE_IN_AREA {},:LOCATION_IN_AREA {},:KNOWS_SN {},:KNOWS {},:CALLER {},:CALLED {},:KNOWS_PHONE {},:OCCURRED_AT {},:INVESTIGATED_BY {},:INVOLVED_IN {},:PARTY_TO {},:FAMILY_REL {rel_type: STRING},:KNOWS_LW {}" Relationship point from source to target nodes "(:Person)-[:FAMILY_REL]->(:Person),(:Person)-[:CURRENT_ADDRESS]->(:Location),(:Person)-[:KNOWS]->(:Person),(:Person)-[:HAS_PHONE]->(:Phone),(:Person)-[:KNOWS_PHONE]->(:Person),(:Person)-[:HAS_EMAIL]->(:Email),(:Person)-[:KNOWS_SN]->(:Person),(:Person)-[:KNOWS_LW]->(:Person),(:Person)-[:PARTY_TO]->(:Crime),(:Location)-[:LOCATION_IN_AREA]->(:Area),(:Location)-[:HAS_POSTCODE]->(:PostCode),(:PostCode)-[:POSTCODE_IN_AREA]->(:Area),(:PhoneCall)-[:CALLED]->(:Phone),(:PhoneCall)-[:CALLER]->(:Phone),(:Crime)-[:INVESTIGATED_BY]->(:Officer),(:Crime)-[:OCCURRED_AT]->(:Location),(:Object)-[:INVOLVED_IN]->(:Crime),(:Vehicle)-[:INVOLVED_IN]->(:Crime)"',
    "nlq_5": "List the addresses with crimes commited in the last month.",
    "schema_5": 'Node properties are the following: ":Person {surname: STRING, nhs_no: STRING, name: STRING, age: STRING},:Location {latitude: FLOAT, postcode: STRING, longitude: FLOAT, address: STRING},:Phone {phoneNo: STRING},:Email {email_address: STRING},:Officer {badge_no: STRING, rank: STRING, name: STRING, surname: STRING},:PostCode {code: STRING},:Area {areaCode: STRING},:PhoneCall {call_duration: STRING, call_time: STRING, call_date: STRING, call_type: STRING},:Crime {date: STRING, id: STRING, type: STRING, last_outcome: STRING, note: STRING, charge: STRING},:Object {description: STRING, id: STRING, type: STRING},:Vehicle {model: STRING, reg: STRING, make: STRING, year: STRING}" Relationship properties are the following: ":CURRENT_ADDRESS {},:HAS_PHONE {},:HAS_EMAIL {},:HAS_POSTCODE {},:POSTCODE_IN_AREA {},:LOCATION_IN_AREA {},:KNOWS_SN {},:KNOWS {},:CALLER {},:CALLED {},:KNOWS_PHONE {},:OCCURRED_AT {},:INVESTIGATED_BY {},:INVOLVED_IN {},:PARTY_TO {},:FAMILY_REL {rel_type: STRING},:KNOWS_LW {}" Relationship point from source to target nodes "(:Person)-[:FAMILY_REL]->(:Person),(:Person)-[:CURRENT_ADDRESS]->(:Location),(:Person)-[:KNOWS]->(:Person),(:Person)-[:HAS_PHONE]->(:Phone),(:Person)-[:KNOWS_PHONE]->(:Person),(:Person)-[:HAS_EMAIL]->(:Email),(:Person)-[:KNOWS_SN]->(:Person),(:Person)-[:KNOWS_LW]->(:Person),(:Person)-[:PARTY_TO]->(:Crime),(:Location)-[:LOCATION_IN_AREA]->(:Area),(:Location)-[:HAS_POSTCODE]->(:PostCode),(:PostCode)-[:POSTCODE_IN_AREA]->(:Area),(:PhoneCall)-[:CALLED]->(:Phone),(:PhoneCall)-[:CALLER]->(:Phone),(:Crime)-[:INVESTIGATED_BY]->(:Officer),(:Crime)-[:OCCURRED_AT]->(:Location),(:Object)-[:INVOLVED_IN]->(:Crime),(:Vehicle)-[:INVOLVED_IN]->(:Crime)"',
}

In [14]:
for i in range(1,6):
    nlq = eval_dict.get(f"nlq_{i}")
    out = prompt_tok(nlq, eval_dict.get(f"schema_{i}"))
    print(nlq, '\n\n', out.strip())
    print("================================================================")

Finds all phone calls lasting more than 60 minutes 

 MATCH (n:PhoneCall)-[r:HAS_CALL]->(m:PhoneCall) WHERE toInteger(n.call_duration) > 60 RETURN n, r, m</s>
Find cars involved in crime with ID '%crime_id%'. 

 MATCH (n:Crime)-[r:INVOLVED_IN]->(m:Vehicle) WHERE toLower(n.id) = toLower('%crime_id%') RETURN n, r, m</s>
List all persons who know someone named Abi. 

 MATCH (n:Person)-[r:KNOWS]->(m:Person) WHERE toLower(m.name) = toLower('Abi') RETURN n, r, m</s>
Find all Officers whose names contain 'Alex', along with their associated entities 

 MATCH (n:Officer)-[r:KNOWS]->(m:Person) WHERE toLower(n.name) CONTAINS toLower('Alex') RETURN n, r, m</s>
List the addresses with crimes commited in the last month. 

 MATCH (n:Crime)-[r:OCCURRED_AT]->(l:Location) WHERE toLower(l.address) CONTAINS toLower('crimes') RETURN n, r, l</s>


In [10]:
test = "who are you?"
prompt_tok(test, schema3)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


'> "I am a Neo4j node with properties: {surname: \'Smith\', name: \'John\', age: \'30\', nhs_no: \'123456789\', badge_no: \'ABC123\', rank: \'Senior\', postcode: \'AB1 1BB\', longitude: \'1.23\', latitude: \'4.56\', address: \'123 Main St\', areaCode: \'AB1\', call_duration: \'10\', call_time: \'09:00\', call_date: \'2022-01-01\', call_type: \'Incident\', description: \'A car was involved in a crime\', id: \'123\', type: \'Crime\', last_outcome: \'Mitigation\', note: \'The car was involved in a theft\', charge: \'Theft\'}"</s>'

In [17]:
prompt(nlq, schema)

'You are an assistant that takes a natural language query (NLQ) and a graph database schema to produce a Neo4J Cypher query.\n\n    ### Instruction:\n\n    NLQ: "Finds all phone calls lasting more than 60 minutes" \n    \n    Schema: Node properties are the following: ":Person {surname: STRING, nhs_no: STRING, name: STRING, age: STRING},:Location {latitude: FLOAT, postcode: STRING, longitude: FLOAT, address: STRING},:Phone {phoneNo: STRING},:Email {email_address: STRING},:Officer {badge_no: STRING, rank: STRING, name: STRING, surname: STRING},:PostCode {code: STRING},:Area {areaCode: STRING},:PhoneCall {call_duration: STRING, call_time: STRING, call_date: STRING, call_type: STRING},:Crime {date: STRING, id: STRING, type: STRING, last_outcome: STRING, note: STRING, charge: STRING},:Object {description: STRING, id: STRING, type: STRING},:Vehicle {model: STRING, reg: STRING, make: STRING, year: STRING}" Relationship properties are the following: ":CURRENT_ADDRESS {},:HAS_PHONE {},:HAS_EMA

## Homework - Token Check

Optional Homework!  Check that there aren't any token issues https://hamel.dev/notes/llm/finetuning/05_tokenizer_gotchas.html